In [24]:
from io import BytesIO
import pycurl,timeit
from multiprocessing import Process
from multiprocessing import Manager

# Performance with proxy

In [25]:
def proxy_latency(prox_type,host,port,target,return_list):
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(pycurl.PROXY, f"{host}:{port}")
    c.setopt(pycurl.URL, target)
    c.setopt(c.WRITEDATA, buffer)
    ts = timeit.default_timer()
    c.perform()
    
    m = {}
    m['prox_type'] = prox_type
    m['target'] = target
    m['total-time'] = c.getinfo(pycurl.TOTAL_TIME)
    m['namelookup-time'] = c.getinfo(pycurl.NAMELOOKUP_TIME)
    m['connect-time'] = c.getinfo(pycurl.CONNECT_TIME)
    m['pretransfer-time'] = c.getinfo(pycurl.PRETRANSFER_TIME)
    m['starttransfer-time'] = c.getinfo(pycurl.STARTTRANSFER_TIME)
    
    return_list.append(m)
    
    c.close()
    
    return m


# Performance without proxy

In [26]:
def normal_latency(port,target,return_list):
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(pycurl.URL, target)
    c.setopt(c.WRITEDATA, buffer)
    ts = timeit.default_timer()
    c.perform()
    
    m = {}
    m['prox_type'] = "N/A"
    m['target'] = target
    m['total-time'] = c.getinfo(pycurl.TOTAL_TIME)
    m['namelookup-time'] = c.getinfo(pycurl.NAMELOOKUP_TIME)
    m['connect-time'] = c.getinfo(pycurl.CONNECT_TIME)
    m['pretransfer-time'] = c.getinfo(pycurl.PRETRANSFER_TIME)
    m['starttransfer-time'] = c.getinfo(pycurl.STARTTRANSFER_TIME)
    
    return_list.append(m)
    
    c.close()
    

# Setup

In [27]:
host = "130.64.148.78"
port = "8080"
range_k = 100



# target = "https://github.com/"

target = "http://www.cs.cmu.edu/~prs/bio.html"


# target = "http://www.cs.tufts.edu/comp/112/"

# Without proxy

## 1. One time

In [28]:
return_list = []
normal_latency(port,target,return_list)
print(return_list[0])

{'prox_type': 'N/A', 'target': 'http://www.cs.cmu.edu/~prs/bio.html', 'total-time': 0.075918, 'namelookup-time': 0.038617, 'connect-time': 0.055892, 'pretransfer-time': 0.055956, 'starttransfer-time': 0.075824}


## 2. Average time (100 request)

In [29]:
total = 0
return_list = []
for n in range(range_k):
    result = normal_latency(port,target,return_list)

for result in return_list: 
    total+= result["total-time"]
    
total/range_k

0.04770776999999998

# Our project - select + pthread

In [30]:
prox_type = "select"

## 1. One time without cache

In [31]:
return_list = []
proxy_latency(prox_type,host,port,target,return_list)
print(return_list[0])

{'prox_type': 'select', 'target': 'http://www.cs.cmu.edu/~prs/bio.html', 'total-time': 0.061433, 'namelookup-time': 4.1e-05, 'connect-time': 0.000187, 'pretransfer-time': 0.000222, 'starttransfer-time': 0.06135}


## 2. Average time with cache (100 reqests)

In [32]:
total = 0
return_list = []
for n in range(range_k):
    proxy_latency(prox_type,host,port,target,return_list)

    
for result in return_list: 
    total+= result["total-time"]
    
total/range_k

0.00038585000000000003

# Simulate multi users - multiprocesses

In [33]:
range_k = 100

## Without proxy - http, 50 request

In [34]:
manager = Manager()
return_list = manager.list()
jobs = []
total = 0

for i in range(range_k):
    p = Process(target=normal_latency,args=(port,target,return_list))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

for item in return_list:
    total+=item["total-time"]

total/range_k    

0.07103274000000002

## Proxy - http with cache, 50 request

In [35]:
manager = Manager()
return_list = manager.list()
jobs = []
total = 0

for i in range(range_k):
    p = Process(target=proxy_latency,args=(prox_type,host,port,target,return_list))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

for item in return_list:
    total+=item["total-time"]

total/range_k  

0.0010697299999999996

In [36]:
target = "https://github.com/"
range_k = 200

## Without proxy - https, 50 request

In [37]:
manager = Manager()
return_list = manager.list()
jobs = []
total = 0

for i in range(range_k):
    p = Process(target=normal_latency,args=(port,target,return_list))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

for item in return_list:
    total+=item["total-time"]

total/range_k    

0.6566602400000003

## Proxy - https withoutcache, 50 request

In [38]:
manager = Manager()
return_list = manager.list()
jobs = []
total = 0

for i in range(range_k):
    p = Process(target=proxy_latency,args=(prox_type,host,port,target,return_list))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

for item in return_list:
    total+=item["total-time"]

total/range_k 

3.8339467799999998

In [39]:
# 10: https noproxy: 0.21 select+pthread: 0.33 select: 0.48  
# 50: https noproxy: 0.42 select+pthread: 1.35 select: 2.11  
# 100: https noproxy: 0.78 select+pthread: 2.24 select: 3.32 
# 200: https noproxy: 1.26 select+pthread: 4.12 select: 7.11 

In [40]:
# Single reqest
# https noproxy: 0.17 select+pthread: 0.18 select: 0.21